In [12]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import datetime

from time import sleep
from random import randint
from IPython.core.display import clear_output
import time 

### Get the movie list from year 2015 until 2019. because it is quite slow, just get the data year by year, change the year manually and repeat it 5 times

In [13]:
years_url = [str(i) for i in range(2015,2016)]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)'
           'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# Redeclaring the lists to store data in
uniqueIds = []
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
gross = []

# Preparing the monitoring of the loop
start_time = time.time()
requests_count = 0
row_count = 0

# For every year in the interval 2000-2017
for year_url in years_url:
    
    pages = [int(i) for i in range(1,50)]
    # For every page in the interval 1-4
    for page in pages:
        
        #print(type(page))
        #break
        
        if page == 1:
            start_from = 1
        else:
            start_from = (page*50)+1
        start_from = str(start_from)
        
        # Make a get request
        url = 'http://www.imdb.com/search/title?release_date=' + year_url +'&sort=num_votes,desc&start=' + start_from
        #url = 'https://www.imdb.com/search/title/?genres=action&sort=num_votes,asc&explore=title_type,genres'
        response = requests.get(url, headers=headers)
        
        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests_count += 1
        elapsed_time = time.time() - start_time
        print('requests_count:{}; Frequency: {} requests_count/s'.format(requests_count, requests_count/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests_count, response.status_code))

        
        # Break the loop if the number of requests is greater than expected
        if requests_count > 100:
            warn('Number of requests was greater than expected.')
            break
            
        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')
        
        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
        
        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:
                
                row_count += 1
                
                uniqueId = container.find('img', class_ = 'loadlate')['data-tconst']
                uniqueIds.append(uniqueId)
                
                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year
                #year = container.h3.find('span', class_ = 'lister-item-year').text
                #year = year[-5:-1]
                #years.append(int(year))

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                list_nv = container.find_all('span', attrs = {'name':'nv'})
                for i in range(len(list_nv)):
                    if i == 0:
                        vote = list_nv[i]['data-value']
                        votes.append((vote))
                    else:
                        m_gross = list_nv[i]['data-value'].replace(',', '')
                        gross.append((m_gross))
                        
                if (len(list_nv) == 1):#
                    m_gross = 0
                    gross.append((m_gross))

                if row_count==10:
                    break

requests_count:49; Frequency: 0.061078886337210724 requests_count/s


### combine 5 years data together and generated a csv file.

In [ ]:
df_movieList_2015 = pd.read_csv('2015-imdb.csv', sep=',')
df_movieList_2016 = pd.read_csv('2016-imdb.csv', sep=',')
df_movieList_2017 = pd.read_csv('2017-imdb.csv', sep=',')
df_movieList_2018 = pd.read_csv('2018-imdb.csv', sep=',')
df_movieList_2019 = pd.read_csv('2019-imdb.csv', sep=',')

In [ ]:
df_movieList = df_movieList_2016.append(df_movieList_2017, ignore_index=True)
df_movieList = df_movieList.append(df_movieList_2015, ignore_index=True)
df_movieList = df_movieList.append(df_movieList_2018, ignore_index=True)
df_movieList = df_movieList.append(df_movieList_2019, ignore_index=True)
df_movieList.count

In [ ]:
from datetime import datetime
now = datetime.now()
now = now.strftime("%Y-%m-%d-%H-%M-%S")
filename_imdb_list = "2015-2019-imdb.csv"
df_movieList.to_csv(filename_imdb_list,index=False)

### Get user playlist that generated by user

In [15]:
l_listName = []
l_listId = []
l_UniqueId = []

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)'
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

filename_imdb_list = '2015-2019-imdb.csv'
df_movieList = pd.read_csv(filename_imdb_list, sep=',')
df_movieList.head()

for m in range(len(df_movieList)):
    uniqueId = str(df_movieList['uniqueId'][m])
    
    url = str('http://www.imdb.com/lists/'+uniqueId+'?ref_=ttrv_rls_sm')
    print(url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content,'html.parser')

    lister_container = soup.find_all('div', class_ = 'article listo')
    for container in lister_container:

        list_even = container.find_all('div', class_ = 'list-preview even')
        for j in range(len(list_even)):
            list_name = list_even[j].find('div', class_ = 'list_name').a.text
            list_url = list_even[j].find('div', class_ = 'list_name').a['href']
            list_id = list_url[6:17]
            list_name = " ".join(list_name.split())
            l_listName.append(list_name)
            l_listId.append(list_id)
            l_UniqueId.append(uniqueId)
            

http://www.imdb.com/lists/tt1431045?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt3498820?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt2975590?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt1386697?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt1211837?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt2543164?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt3748528?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt3783958?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt2948356?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt2119532?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt4572514?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt3949660?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt3544112?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt2592614?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt4731136?ref_=ttrv_rls_sm
http://www.imdb.com/lists/tt1974419?ref_=ttrv_rls_sm


KeyboardInterrupt: 

In [3]:
#create imdb-user-list.csv
df_user_list = pd.DataFrame(
                    {'uniqueId': l_UniqueId,
                    'name': l_listName,
                  'listId': l_listId,
                  })
df_user_list.head()

uniqueId                              name       listId
0  tt3498820                       2020 Summer  ls080003802
1  tt3498820                         Películas  ls096174745
2  tt3498820                        MCU Top 23  ls042246793
3  tt3498820                         MARVELous  ls058022803
4  tt3498820  Best Comic Book Movies of Decade  ls080446005

In [4]:
from datetime import datetime
now = datetime.now()
now = now.strftime("%Y-%m-%d-%H-%M-%S")
filename_userlist = "2015-2019-imdb-user-list-"+str(now)+".csv"
df_user_list.to_csv(filename_userlist,index=False)

In [7]:
### crawl full movie list for each playlist that user generated 

In [6]:
l_playlist_uniqueId = []
l_playlist_listId = []
l_playlist_listName = []
l_playlist_userId = []
l_playlist_userName = []

# Preparing the monitoring of the loop
start_time = time.time()
requests_count = 0
row_count = 0

filename = '2015-2019-imdb-user-list.csv'
df_user_list = pd.read_csv(filename, sep=',')

# Parse the content of the request with BeautifulSoup
#page_html = BeautifulSoup(response.text, 'html.parser')
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)'
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

for m in range(len(df_user_list)):
    listId = str(df_user_list['listId'][m])
    listName = str(df_user_list['name'][m])
    
    playlist_url = 'https://www.imdb.com/list/'+listId+'/'
    print((playlist_url))
    responsePlaylist = requests.get(playlist_url, headers=headers)
    soupPlaylist = BeautifulSoup(responsePlaylist.content,'html.parser') 

     # Pause the loop
    sleep(randint(8,15))

    # Monitor the requests
    requests_count += 1
    elapsed_time = time.time() - start_time
    print('requests_count:{}; Frequency: {} requests_count/s'.format(requests_count, requests_count/elapsed_time))
    clear_output(wait = True)
    
    # Throw a warning for non-200 status codes
    if soupPlaylist.status_code != 200:
        print('Request: {}; Status code: {}'.format(requests_count, soupPlaylist.status_code))


    # Break the loop if the number of requests is greater than expected
    if requests_count > 1000:
        print('Number of requests was greater than expected.')
        break
        
    all_playlist_uniqueId = []
    playlist_containers = soupPlaylist.find_all('div', class_ = 'lister-list')
    for playlist_container in playlist_containers:
        playlist = playlist_container.find_all('div', class_ = 'lister-item mode-detail')
        for j in range(len(playlist)):
            playlist_uniqueId = playlist[j].find('div', class_='lister-item-image ribbonize')['data-tconst']
            #print(playlist_uniqueId)
            
            if playlist_uniqueId in all_playlist_uniqueId:
                print(playlist_uniqueId)
            else:
                #all_playlist_uniqueId.append(playlist_uniqueId)
                l_playlist_uniqueId.append(playlist_uniqueId)
                l_playlist_listId.append(listId)
                l_playlist_listName.append(listName)
    


Request: 1001; Status code: None
Number of requests was greater than expected.


In [7]:
#create imdb-playlist.csv
df_playlist = pd.DataFrame(
                    {'uniqueId': l_playlist_uniqueId,
                     'listId': l_playlist_listId,
                     'listName': l_playlist_listName,
                  })
df_playlist.head()

uniqueId       listId     listName
0  tt2930604  ls080003802  2020 Summer
1  tt0319061  ls080003802  2020 Summer
2  tt3748528  ls080003802  2020 Summer
3  tt9071322  ls080003802  2020 Summer
4  tt0076759  ls080003802  2020 Summer

In [8]:
from datetime import datetime
now = datetime.now()
now = now.strftime("%Y-%m-%d-%H-%M-%S")
filename_playlist = "imdb-user-playlist-"+str(now)+".csv"
df_playlist.to_csv(filename_playlist,index=False)